In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/order_items")

df.show()

In [0]:
df_order_items = (
    spark.readStream 
    .format("cloudFiles").option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .option("cloudFiles.schemaHints", "order_item_id INT, price DECIMAL(10,2), freight_value DECIMAL(10,2), shipping_limit_date TIMESTAMP")
    .option("cloudFiles.schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/order_items")
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/order_items/")
    .writeStream
    .format("delta")
    .trigger(availableNow=True)
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/order_items") 
    .table("bronze.order_items")
)

In [0]:
%sql
describe bronze.order_items;

In [0]:
%sql
SELECT * FROM cloud_files_state('abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/order_items');

In [0]:
%sql
select * from bronze.order_items limit 10;